# PoC of attr class saved inside RethinkDB

chilling around with `attrs`.

Source:
https://glyph.twistedmatrix.com/2016/08/attrs.html

Documentation:
https://attrs.readthedocs.io/en/stable/overview.html

## Database preparation

In [1]:
import rethinkdb as r

# variables
host = 'dbr'
db = 'mydb'
table = 'mytable'

In [2]:
r.connect(host=host).repl()

# Watch out

if db in r.db_list().run():
    r.db_drop(db).run()
r.db_create(db).run()

if table not in r.db(db).table_list().run():
    r.db(db).table_create(table).run()


## Class preparation

In [3]:
import attr

@attr.s
class MyClass(object):
    x = attr.ib()
    y = attr.ib()

test = MyClass(x=1, y=2)
test

MyClass(x=1, y=2)

## Save the object

In [4]:
query = r.db(db).table(table)

query.insert(attr.asdict(test)).run()

{'deleted': 0,
 'errors': 0,
 'generated_keys': ['3e8a16f8-b74c-4fa6-80c4-bda7d2a0aa1e'],
 'inserted': 1,
 'replaced': 0,
 'skipped': 0,
 'unchanged': 0}

## Recover the object

In [5]:
args = list(query.run()).pop()
args.pop('id')
test2 = MyClass(**args)

In [6]:
test == test2

True